# 🤖 AI Paragraph Generation (GGUF Models)

**목표**: Human 문단을 로컬 GGUF 모델에 넣어 AI 스타일 문단을 생성하여 (Human, AI) 페어 데이터셋 구축

## 🛠️ 설정
- **사용 가능한 모델**:
  1. HyperCLOVAX 0.5B (한국어 특화)
  2. Qwen3 0.6B (다국어 지원)
  3. Qwen3 1.7B (더 강력한 성능)
- **샘플 수**: 이전 데이터(gpt_oss) 100개 문서 제외 → 새로운 100개 문서 샘플링
- **목표**: ~1,140개 페어 추가 생성

In [1]:
import os
import pandas as pd
from tqdm.auto import tqdm
import time
from datetime import datetime, timedelta
from llama_cpp import Llama

## 🎯 모델 선택

In [2]:
# 모델 선택 옵션 (하나만 선택하세요)
MODEL_CHOICE = "hyperclova"  # 옵션: "hyperclova", "qwen-0.6b", "qwen-1.7b"

# 모델 경로 설정
MODEL_CONFIGS = {
    "hyperclova": {
        "path": os.path.expanduser("~/.cache/huggingface/hub/models--cherryDavid--HyperCLOVA-X-SEED-Vision-Instruct-3B-Llamafied-Q4_K_S-GGUF/snapshots/baadee14ff4bbce6cd5613ef3648deb4181e5ab0/hyperclova-x-seed-vision-instruct-3b-llamafied-q4_k_s-imat.gguf"),
        "name": "HyperCLOVAX 3B",
        "output_suffix": "hyperclova"
    },
    "qwen-0.6b": {
        "path": os.path.expanduser("~/.cache/huggingface/hub/Qwen3-0.6B-GGUF/Qwen3-0.6B-Q4_K_M.gguf"),
        "name": "Qwen3 0.6B",
        "output_suffix": "qwen06b"
    },
    "qwen-1.7b": {
        "path": os.path.expanduser("~/.cache/huggingface/hub/Qwen3-1.7B-GGUF/Qwen3-1.7B-Q8_0.gguf"),
        "name": "Qwen3 1.7B",
        "output_suffix": "qwen17b"
    }
}

# 선택된 모델 정보
model_config = MODEL_CONFIGS[MODEL_CHOICE]
MODEL_PATH = model_config["path"]
MODEL_NAME = model_config["name"]
OUTPUT_SUFFIX = model_config["output_suffix"]

# 모델 파일 존재 확인
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"모델 파일을 찾을 수 없습니다: {MODEL_PATH}")

print(f"📌 선택된 모델: {MODEL_NAME}")
print(f"📂 모델 경로: {MODEL_PATH}")
print(f"💾 출력 파일 접미사: {OUTPUT_SUFFIX}")

📌 선택된 모델: HyperCLOVAX 3B
📂 모델 경로: /Users/youngjinson/.cache/huggingface/hub/models--cherryDavid--HyperCLOVA-X-SEED-Vision-Instruct-3B-Llamafied-Q4_K_S-GGUF/snapshots/baadee14ff4bbce6cd5613ef3648deb4181e5ab0/hyperclova-x-seed-vision-instruct-3b-llamafied-q4_k_s-imat.gguf
💾 출력 파일 접미사: hyperclova


## 📂 데이터 로드 및 중복 제외 샘플링

In [3]:
# Human 문단 데이터 로드
DATA_PATH = '/Users/youngjinson/Downloads/open/human_paragraphs.csv'
para_df = pd.read_csv(DATA_PATH)
print(f"총 문단 수: {len(para_df):,}개")

총 문단 수: 1,125,652개


In [4]:
# 이전에 사용한 doc_id 제외 (gpt_oss에서 사용한 100개 문서)
PREV_DATA_PATH = '/Users/youngjinson/Downloads/open/gpt_oss_synthetic_pairs_cleaned.csv'
prev_df = pd.read_csv(PREV_DATA_PATH)
used_doc_ids = set(prev_df['source_doc_id'].unique())
print(f"이전에 사용한 문서 수: {len(used_doc_ids)}개")

# 사용되지 않은 doc_id만 필터링
available_df = para_df[~para_df['source_doc_id'].isin(used_doc_ids)].copy()
print(f"사용 가능한 문단 수: {len(available_df):,}개")

이전에 사용한 문서 수: 100개
사용 가능한 문단 수: 1,124,478개


In [5]:
# 새로운 100개 문서 샘플링
N_DOCS = 100
SEED = 43  # 이전과 다른 시드

unique_doc_ids = available_df['source_doc_id'].unique()
sampled_doc_ids = pd.Series(unique_doc_ids).sample(n=min(N_DOCS, len(unique_doc_ids)), random_state=SEED).values
sampled_df = available_df[available_df['source_doc_id'].isin(sampled_doc_ids)].copy()
sampled_df = sampled_df.sort_values(['source_doc_id', 'para_index']).reset_index(drop=True)

print(f"샘플링: {len(sampled_doc_ids)}개 문서, {len(sampled_df)}개 문단")

샘플링: 100개 문서, 1126개 문단


## 🤖 GGUF 모델 로드

In [6]:
# GGUF 모델 로드
print(f"🔄 {MODEL_NAME} 로딩 중...")
llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=2048,  # 컨텍스트 길이
    n_threads=4,  # CPU 스레드 수
    verbose=False
)
print(f"✅ {MODEL_NAME} 로드 완료!")

🔄 HyperCLOVAX 3B 로딩 중...


llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64

✅ HyperCLOVAX 3B 로드 완료!


## ✍️ 생성 함수

In [7]:
PROMPT_TEMPLATE = """다음 문단을 AI 스타일로 다시 작성해주세요. 
원본의 정보와 뜻은 유지하되, 자연스러운 AI 생성 텍스트처럼 작성해주세요. 종결어미는 원본을 따라해주세요.
추가 설명 없이 변환된 문단만 출력해주세요.

원본 문단:
{paragraph}

AI 스타일 문단:"""

def generate_ai_paragraph(paragraph, max_retries=3):
    """GGUF 모델을 사용하여 AI 스타일 문단 생성"""
    prompt = PROMPT_TEMPLATE.format(paragraph=paragraph)
    
    for attempt in range(max_retries):
        try:
            output = llm(
                prompt,
                max_tokens=512,
                temperature=0.7,
                top_p=0.9,
                stop=["\n\n", "원본 문단:"],
                echo=False
            )
            response = output['choices'][0]['text'].strip()
            if response:
                return response
        except Exception as e:
            print(f"Error (attempt {attempt+1}): {e}")
            time.sleep(2 ** attempt)
    return None

## 🔄 1개씩 생성 + 저장 + 출력

In [8]:
OUTPUT_PATH = f'/Users/youngjinson/Downloads/open/{OUTPUT_SUFFIX}_synthetic_pairs.csv'

# 기존 결과 로드 (이어서 하기 위함)
if os.path.exists(OUTPUT_PATH):
    result_df = pd.read_csv(OUTPUT_PATH)
    results = result_df.to_dict('records')
    completed_ids = set(result_df['para_id'].tolist())
    print(f"📂 기존 데이터 로드: {len(results)}개 완료")
else:
    results = []
    completed_ids = set()
    print("🆕 새로 시작")

print(f"남은 작업: {len(sampled_df) - len(completed_ids)}개")

📂 기존 데이터 로드: 61개 완료
남은 작업: 1065개


In [ ]:
# 생성 실행
start_time = time.time()
times = []

for idx, row in sampled_df.iterrows():
    para_id = row.name if 'para_id' not in row else row['para_id']
    
    if para_id in completed_ids:
        continue
    
    iter_start = time.time()
    
    ai_paragraph = generate_ai_paragraph(row['paragraph'])
    
    result = {
        'para_id': para_id,
        'source_doc_id': row['source_doc_id'],
        'para_index': row['para_index'],
        'source_title': row.get('source_title', ''),
        'human_paragraph': row['paragraph'],
        'ai_paragraph': ai_paragraph,
        'generation_success': ai_paragraph is not None,
        'model_name': MODEL_NAME
    }
    results.append(result)
    completed_ids.add(para_id)
    
    # 매번 저장 (체크포인트)
    pd.DataFrame(results).to_csv(OUTPUT_PATH, index=False)
    
    # 진행 상황 출력
    elapsed = time.time() - iter_start
    times.append(elapsed)
    avg_time = sum(times) / len(times)
    remaining = len(sampled_df) - len(completed_ids)
    eta = timedelta(seconds=int(avg_time * remaining))
    
    print(f"[{len(completed_ids)}/{len(sampled_df)}] {elapsed:.1f}초 | ETA: {eta}")
    print(f"  📝 {row.get('source_title', '')[:20]}...")
    print(f"  {'✅ 성공' if ai_paragraph else '❌ 실패'}")
    print()

print(f"\n🎉 완료! 총 {len(results)}개 생성")

[62/1126] 16.9초 | ETA: 4:59:35
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[63/1126] 11.4초 | ETA: 4:10:28
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[64/1126] 33.9초 | ETA: 6:06:50
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[65/1126] 12.1초 | ETA: 5:28:17
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[66/1126] 15.2초 | ETA: 5:16:07
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[67/1126] 51.6초 | ETA: 6:54:51
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[68/1126] 10.4초 | ETA: 6:21:20
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[69/1126] 9.5초 | ETA: 5:54:18
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[70/1126] 15.5초 | ETA: 5:45:00
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[71/1126] 7.2초 | ETA: 5:22:50
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[72/1126] 15.9초 | ETA: 5:18:35
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[73/1126] 14.4초 | ETA: 5:12:46
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[74/1126] 10.7초 | ETA: 5:02:53
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[75/1126] 11.1초 | ETA: 4:54:55
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성공

[76/1126] 12.6초 | ETA: 4:49:40
  📝 필리핀 전역 (1944년~1945년)...
  ✅ 성

## 🧹 중복 제거 및 정리

In [ ]:
# 중복 제거 (Human == AI인 경우)
df = pd.read_csv(OUTPUT_PATH)
print(f"원본: {len(df)}개")

df_clean = df[df['human_paragraph'] != df['ai_paragraph']].copy()
print(f"중복 제거 후: {len(df_clean)}개 ({len(df) - len(df_clean)}개 제거)")

# 정제된 파일 저장
CLEANED_PATH = f'/Users/youngjinson/Downloads/open/{OUTPUT_SUFFIX}_synthetic_pairs_cleaned.csv'
df_clean.to_csv(CLEANED_PATH, index=False)
print(f"✅ 저장 완료: {CLEANED_PATH}")